In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torchsummary import summary

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
no_cuda = False
seed = 1
log_interval = 10
torch.manual_seed(seed)
device = torch.device("cpu")

In [4]:
def train(model, device, train_loder, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loder):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loder.dataset), 100. * batch_idx / len(train_loder), loss.item()))

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(),
                                                              transforms.Normalize((0.1307,), (0.3081,))])),
                                           batch_size=batch_size, shuffle=True)

In [7]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, 
                                                         transform=transforms.Compose([
                                                             transforms.ToTensor(),
                                                             transforms.Normalize((0.1307,), (0.3081,))])),
                                          batch_size=test_batch_size, shuffle=True)

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [9]:
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2             [-1, 20, 8, 8]           5,020
         Dropout2d-3             [-1, 20, 8, 8]               0
            Linear-4                   [-1, 50]          16,050
            Linear-5                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.08
Estimated Total Size (MB): 0.15
----------------------------------------------------------------


In [10]:
for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.386960
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.322700
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.308476
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.278819
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.230376
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.237482
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.198882
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.143278
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.163671
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.962688
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.035229
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.817211
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.791994
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.699615
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.464242
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.609711
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.308749
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.345274
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.202445
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.330626
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.497581
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.447419
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.323874
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.311617
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.307132
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.373376
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.140742
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.155251
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.360286
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.327369
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.252565
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.303580
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.357752
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.317675
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.245111
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.399602
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.463665
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.410724
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.455307
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.308275


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.292531
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.392413
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.305745
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.186541
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.172375
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.240054
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.290310
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.118163
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.160691
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.217497
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.268485
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.325080
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.244211
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.291384
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.136965
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.415146
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.275717
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.123156
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.204073
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.231356


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.163099
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.274583
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.178949
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.144280
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.106579
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.201458
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.275397
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.150430
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.248401
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.211369
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.291831
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.430315
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.224967
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.237514
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.454866
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.242027
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.268860
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.290427
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.155607
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.228173


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.363575
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.147499
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.373916
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.250964

Test set: Average loss: 0.0615, Accuracy: 9808/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.266146
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.351874
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.208587
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.273649
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.236012
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.261050
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.171975
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.241222
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.228250
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.098472
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.282670
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.092181
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.085672
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.142092
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.091436
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.125352
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.275457
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.099058
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.134532
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.108624
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.281743
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.256643
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.090552
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.126541
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.201866
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.077808
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.265076
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.226172
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.119025
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.112967
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.213255
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.205288
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.152745
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.155833
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.219252
